In [ ]:
%load_ext autoreload
%autoreload 2
import globvars as gv
from globvars import gp

In [ ]:
#$RS --mods msrc --feat_types H_act --parcel_group_names Sensorimotor,OccipitalI    nf,FrontalSup,FrontalInf,TemporalMid  --prefix onlyH_act_SMyOIyFSyFIyTM_noLFP

# Set params

In [ ]:
# normally I want to join all datasets within each subject (across medcond and task)
inc_S03 = True
inc_noMvt = False
join_medcond = False  # medconds merged across subj
join_subjects = False

#overlapping_windows = True
overlapping_windows = False
semidisjoint_windows = True
disjoint_windows = True
exCB_sides = True   # here we use only LFP anyway
incCB_sides = True

rawnames_dict = {}

prefixes_types = ['PREFIXES_H']
prefixes_types = ['PREFIXES_H_LFP_SIDED']
prefixes_types += ['PREFIXES_H_CTXLFP_SIDED']

#fname_runstr = '_runstrings_ML_test.txt'
fname_runstr = '_runstrings_ML_searchLFP.txt'
g_it_strs = []
g_it_strs += [' --groupings_to_use merge_all_not_trem --int_types_to_use basic']
g_it_strs += [' --groupings_to_use merge_nothing   --int_types_to_use basic']
g_it_strs += [' --groupings_to_use merge_movements --int_types_to_use basic']
g_it_strs += [' --groupings_to_use merge_nothing   --int_types_to_use trem_vs_quiet']

comment = 'Select best LFP using H act on both sides, no XGB tuning'

runpars = {}
runpars['inc_S03'] = inc_S03
runpars['join_subjects'] =join_subjects
runpars['inc_noMvt'] = inc_noMvt
runpars['join_medcond'] = join_medcond
runpars['prefixes_types'] = prefixes_types
runpars['fname_runstr'] = fname_runstr
#runpars['barain_area_inclusion_modes'] = modes
runpars['run_specific_args'] = ' --XGB_tune_param 0'  #techincal params about inclusion of some algortihms or no
runpars['comment'] = comment

# Apply params

In [ ]:
rawnames_dict['off_Mvt_best'] = ['S01_off_hold',
 'S01_off_move',
 'S02_off_hold',
 'S02_off_move',
 'S04_off_hold',
 'S04_off_move',
 'S05_off_hold',
 'S05_off_move',
 'S07_off_hold',
 'S07_off_move']
rawnames_dict['off_unsure'] = ['S03_off_hold', 'S03_off_move']
rawnames_dict['off_Mvt'] = rawnames_dict['off_Mvt_best'] + rawnames_dict['off_unsure']
rawnames_dict['on_Mvt'] = ['S01_on_hold',
 'S01_on_move',
 'S02_on_hold',
 'S02_on_move',
 'S04_on_hold',
 'S04_on_move',
 'S05_on_hold',
 'S05_on_move',
 'S07_on_hold',
 'S07_on_move'] 

rns = rawnames_dict['off_Mvt_best'] + rawnames_dict['on_Mvt']
z = zip(rns[::2], rns[1::2])
per_subj_tasks_merged = list(z)
per_subj_tasks_merged = [ rawnames_dict['off_unsure'] ] + per_subj_tasks_merged
##############################################
rawnames_list = []
if join_subjects:
    if not join_medcond:
        if inc_S03:
            rawnames_list += [ ('normal', rawnames_dict['off_Mvt'] ) ] 
        else:
            rawnames_list += [ ('normal', rawnames_dict['off_Mvt_best'] ) ] 
        if inc_noMvt:
            raise ValueError('not implemented')

        rawnames_list += [ ('normal',rawnames_dict['on_Mvt']) ] 

        if inc_noMvt:
            raise ValueError('not implemented')
    else:
        if inc_S03:
            kk = 'off_Mvt'
        else:
            kk = 'off_Mvt_best'
        rns = rawnames_dict[kk] + rawnames_dict['on_Mvt']

        rawnames_list += [ ('normal',rns ) ] 
else:
    if not join_medcond:
        for tpl in per_subj_tasks_merged:
            rawnames_list += [ ('normal', list(tpl) ) ] 
    else:
        from utils_preproc import getRawnameListStructure
        rnstruct,rnstruct_glob = getRawnameListStructure(list(set(sum(rawnames_dict.values(),[]) )), ret_glob=1)
        for subj,v in rnstruct.items():
            rawnames_list += [ ('normal', list(sorted(v['datasets']))  )  ]
            #s = ','.join(v['datasets'])
            #print(s)

##############################################
if join_subjects:
    if join_medcond:
        rn_test =  ('test',[ 'S01_off_hold', 'S04_off_hold', 'S05_off_move'] ) 
    else:
        rn_test =  ('test',[ 'S01_off_hold', 'S01_on_move', 'S04_off_hold', 'S05_on_move'] ) 
else:
    if not join_medcond:
        rn_test =  ('test',[ 'S04_off_hold', 'S04_off_move'] ) 
    else:
        rn_test =  ('test',[ 'S04_off_hold', 'S04_on_move'] ) 
rawnames_list += [rn_test]
##############################################
pfstr_per_rnt = {}

if join_subjects:
    raise ValueError("not implemented")
    if join_medcond:
        pfile_str = ' --param_file ML_joint2_one_LFP_HPC.ini'
        #TODO: create a new pfile for testing, this one does not exit
        pfile_str_test = ' --param_file ML_joint2_one_LFP_HPC_fast.ini'
    else:
        pfile_str = ' --param_file ML_joint_one_LFP_HPC.ini'
        pfile_str_test = ' --param_file ML_joint_one_LFP_HPC_fast.ini' 
else:
    if not join_medcond:
        #raise ValueError('not implemented')
        pfile_str      = ' --param_file ML_searchLFP_both_HPC.ini'
        pfile_str_test = ' --param_file ML_searchLFP_both_HPC_fast.ini' 
    else:
        pfile_str      = ' --param_file ML_searchLFP_both_HPC.ini'
        pfile_str_test = ' --param_file ML_searchLFP_both_HPC_fast.ini' 
    
pfstr_per_rnt ['normal'] = pfile_str
pfstr_per_rnt ['test']   = pfile_str_test


runpars['incCB_sides'] = incCB_sides
runpars['exCB_sides'] = exCB_sides
runpars['inc_disjoint_windows'] = disjoint_windows
runpars['inc_overlapping_windows'] = overlapping_windows
runpars['prefixes_types'] = prefixes_types
#############################################
BANDS_BETA = 'beta'
BANDS_GAMMA = 'gamma'
BANDS_TREMOR = 'tremor'

rslist_cur = []
if "PREFIXES_H" in prefixes_types:    
    if disjoint_windows:
        rslist_cur += [('modLFP_onlyH_act_disjoint',          '--mods LFP --feat_types H_act --subskip_fit 8')]        
        rslist_cur += [('modLFP_onlyH_disjoint',               '--mods LFP --feat_types H_act,H_mob,H_compl  --subskip_fit 8')] 
    if semidisjoint_windows:
        rslist_cur += [('modLFP_onlyH_act_semidisjoint',          '--mods LFP --feat_types H_act --subskip_fit 4')]        
        rslist_cur += [('modLFP_onlyH_semidisjoint',               '--mods LFP --feat_types H_act,H_mob,H_compl  --subskip_fit 4')]         
    if overlapping_windows:
        rslist_cur += [('modLFP_onlyH_act',           '--mods LFP --feat_types H_act')]        
        rslist_cur += [('modLFP_onlyH',               '--mods LFP --feat_types H_act,H_mob,H_compl')] 
    
if "PREFIXES_H_LFP_SIDED" in prefixes_types:
    side_pairs = [('both','both')]
    if incCB_sides:
        side_pairs += [('left','left'), ('right','right')]
    for side1,side2 in side_pairs:
        addstr = f' --brain_side_to_use {side1} --LFP_side_to_use {side2}'
        if disjoint_windows:
            rslist_cur += [(f'modLFP_onlyH_act_brain{side1}_disjoint',  f'--mods LFP --feat_types H_act --subskip_fit 8'  + addstr)]        
            rslist_cur += [(f'modLFP_onlyH_brain{side1}_disjoint',      f'--mods LFP --feat_types H_act,H_mob,H_compl --subskip_fit 8' + addstr)] 
        if semidisjoint_windows:
            rslist_cur += [(f'modLFP_onlyH_act_brain{side1}_semidisjoint',  f'--mods LFP --feat_types H_act --subskip_fit 4'  + addstr)]        
            rslist_cur += [(f'modLFP_onlyH_brain{side1}_semidisjoint',      f'--mods LFP --feat_types H_act,H_mob,H_compl --subskip_fit 4' + addstr)] 
        if overlapping_windows:
            rslist_cur += [(f'modLFP_onlyH_act_brain{side1}',           f'--mods LFP --feat_types H_act' + addstr )]        
            rslist_cur += [(f'modLFP_onlyH_brain{side1}',               f'--mods LFP --feat_types H_act,H_mob,H_compl' + addstr)]         
            
if "PREFIXES_H_CTXLFP_SIDED" in prefixes_types:
    side_pairs = [('both','both')]
#     if incCB_sides:
#         side_pairs += [('left','left'), ('right','right')]
    if exCB_sides:
        side_pairs += [ ('left_exCB','left'), ('right_exCB','right') ]
    for side1,side2 in side_pairs:
        addstr = f' --brain_side_to_use {side1} --LFP_side_to_use {side2}'
        if disjoint_windows:
            rslist_cur += [(f'onlyH_act_brain{side1}_disjoint',  f'--mods LFP,msrc --feat_types H_act --subskip_fit 8'  + addstr)]        
            rslist_cur += [(f'onlyH_brain{side1}_disjoint',      f'--mods LFP,msrc --feat_types H_act,H_mob,H_compl --subskip_fit 8' + addstr)] 
        if semidisjoint_windows:
            rslist_cur += [(f'onlyH_act_brain{side1}_semidisjoint',  f'--mods LFP,msrc --feat_types H_act --subskip_fit 4'  + addstr)]        
            rslist_cur += [(f'onlyH_brain{side1}_semidisjoint',      f'--mods LFP,msrc --feat_types H_act,H_mob,H_compl --subskip_fit 4' + addstr)]     
        if overlapping_windows:
            rslist_cur += [(f'onlyH_act_brain{side1}',           f'--mods LFP,msrc --feat_types H_act' + addstr )]        
            rslist_cur += [(f'onlyH_brain{side1}',               f'--mods LFP,msrc --feat_types H_act,H_mob,H_compl' + addstr)] 
        
    
    #rslist_cur += ['--feat_types H_act --mods LFP      --prefix onlyH_act_onlyLFP         ']        
    #rslist_cur += ['--feat_types H_act --mods LFP      --prefix onlyH_act_onlyLFP         ']    
assert len(rslist_cur) == len(set(rslist_cur))  # I don't want reapeats
prefs = list( list( zip(*rslist_cur) )[0] )
assert len(prefs) == len(set(prefs))  # I don't want reapeats even in prefs

In [ ]:
#rawnames_list,rslist_cur

# Set timestamp

In [ ]:
import time
import datetime
print(f'Set new timestamp at {datetime.datetime.now()}')
run_corresp_id = int( time.time() * 10000 )


In [ ]:
#rawnames_list

# Generate file

In [ ]:
#prefixes_types, pfstr_per_rnt, g_it_strs, rslist_cur

In [ ]:
save = True
special_globinds = []  
# these are globinds that don't take into account test strings that will be added in the beginning

runstrings_per_rnt = {}
    
n_testrunstr_to_include = 1
runfile = 'run_ML.py' 
no_LFP_unless_explicit = 1
    
global_ind = 0
runstrings = []

for g_it_str in g_it_strs:
    pref2pgn = {}
    for rnt in pfstr_per_rnt:
        runstrings_per_rnt[rnt ] = []
    # Cycle over rawnames
    for rnt,rawnames in rawnames_list:
        rawstr = ','.join(rawnames)    
        for tpl in rslist_cur:  #prefix name, prefix param
            prefix,rs = tpl[:2]
            s = runfile
            s+= f' -r {rawstr}' 
            s+= f' --prefix {prefix}'
            s+= ' ' + rs
            s+= pfstr_per_rnt[rnt]  # param file
            s+= g_it_str
            s+= f' --runCID {run_corresp_id}'
            s+= ' ' + runpars['run_specific_args']                 
            descr = None
            if len(tpl) > 2:
                descr = tpl[2]
            pref2pgn[prefix] = [None,None,descr]
            
            if rnt == 'test':
                s = s.replace('--subskip_fit 8','')

            if s in runstrings_per_rnt[rnt]:
                raise ValueError('repeat')
            runstrings_per_rnt[rnt ] += [s]; 
            if rnt != 'test':
                global_ind += 1            
        #runstrings += runstrings_per_rnt['normal' ]
        
    runstrings += runstrings_per_rnt['normal' ]
    
if n_testrunstr_to_include > 0:
    runstrings = runstrings_per_rnt['test'][:n_testrunstr_to_include] + runstrings

srs = set(runstrings)
assert len(srs) == len(runstrings),  f'there are repeating runstrings {len(srs)} , {len(runstrings)}'
assert len(runstrings), 'Got zero runstrings'

####################
        
from os.path import join as pjoin
import json

if save:
    fname_full_runstr = pjoin(gv.code_dir, 'run', fname_runstr)
    with open( fname_full_runstr, 'w' ) as f:
        for s in runstrings:
            f.write(s + '\n')
    fname_full_runstr_archive = pjoin(gv.code_dir, 'run', f'__{fname_runstr[:-4]}_{run_corresp_id}.json')
    with open( fname_full_runstr, 'w' ) as f:
        for s in runstrings:
            f.write(s + '\n')
            
    print(fname_full_runstr)
    print(fname_full_runstr_archive)

c = {'correspondance':pref2pgn, 'prefixes_types':prefixes_types, 
         'param_dir_str':pfstr_per_rnt ['normal'], 'rawnames_list':rawnames_list,
        'comment':comment, 'runpars':runpars, 'runstrings':runstrings}
if save:
    fn = pjoin(gv.code_dir, 'run', f'___run_corresp_{run_corresp_id}.txt')    
    with open(fn , 'w' ) as f:
        json.dump(c, f, indent =2)
#     f.write(json.dumps(pref2pgn))

    print(fn)
    print(f'Saved res for run_corresp_id = {run_corresp_id}')

In [ ]:
len(runstrings)

In [ ]:
runstrings_per_rnt['test'][0]

In [ ]:
g_it_str,rawstr,tpl

In [ ]:
# vi numbers
110 - 

In [ ]:
inds_per_substrings = {}

In [ ]:
inds_per_substrings

In [ ]:
len( runstrings_per_rnt['normal'] )

In [ ]:
mr = map(int, ','.join( list( inds_per_substrings.values() ) ).split(',') )
mr = list(mr)
mr_ext = mr[:]
for m in mr:
    m2 = m + maxjobs
    if m < maxjobs and m2 not in mr_ext:
        mr_ext += [m2]
not_submitted = list(sorted(  set(range(len(runstrings) ) ) - set(mr_ext)  ))

In [ ]:
len(mr_ext), len(mr), len(runstrings)

In [ ]:
print (','.join( map(str,not_submitted) ) )

In [ ]:
#16:26
#substrings = ['_disjoint', '--mods LFP ' ] #, 'S04']
#substrings = ['_semidisjoint', '--mods LFP ' ] #, 'S04']
#substrings = ['_disjoint', '--mods LFP,msrc ' ] #, 'S04']
substrings = ['_semidisjoint', '--mods LFP,msrc ' ] #, 'S04']
inds = []
maxjobs = 256
small_inds_only = False
for rsi,rs in enumerate(runstrings):
    if rsi == 0:  # test index
        continue
    found_all = True
    for ss in substrings:
        if rs.find(ss) < 0:
            found_all = False
            break
    if found_all:
        if rsi > maxjobs and small_inds_only:
            rsi = rsi % maxjobs
        inds += [rsi]
uinds = sorted(set(inds) )
print( len( uinds) )
runstrings_subsel = np.array(runstrings)[inds].tolist()
s = ','.join( map(str,uinds  ) )
inds_per_substrings[tuple(substrings)] = s
print(s)
display( runstrings_subsel  )
#[i for i in range(len(runstrings)) if runstrings[i].find('_disjoint') >= 0 ]

In [ ]:
runstrings_per_rnt['test'][:n_testrunstr_to_include]

In [ ]:
np.histogram(runstrings)

In [ ]:
runstrings

In [ ]:
prefixes_types

## find inds to recalc

In [ ]:
#to_recalc = [('S07_on', 'OccipitalMid')]
#to_recalc += [('S01_off', 'SupraMarginal')]
to_recalc = [('S01_off', 'SupraMarginal')]
to_recalc += [('S01_off',   'onlyH_act_LFPand_only13') ]# 'onlyH_act_LFPand_only13')]
#'S01_off': {'onlyH_act_LFPand_only13'}
to_recalc = [('S02_on',   'onlyH_act_only15') ]

rsis = []
for rn,name in to_recalc:
    key_founds = []
    for key,item in pref2pgn.items():
        if item[1] == name:
            key_founds += [key]
    if not len(key_founds) and (name in pref2pgn):
        key_founds = [name]
        roi = pref2pgn[name][1]
    else:
        roi = name
    print('key_founds = ', key_founds)
        
    for rsi,rs in enumerate(runstrings):
        if rs.find('_test') >= 0 or rs.find('_fast') >= 0:
            continue
        items = rs.split()
        a = items[1::2]
        b = items[2::2]
        d = dict( zip(a,b) )
        
        c1 = d['-r'].find(rn) >= 0 
        c2 = d['--prefix'] in key_founds
        
        if c1 and c2:
            rsis += [rsi]
            print(f'{rsi:4} = {rn} : {roi:18} -> {d["--prefix"]}')
            #print(rsi)
            
rsis = list(sorted(set(rsis)))
print('\nto be given to sbatch: ', rsis)

In [ ]:
import gc; gc.collect()

In [ ]:
key_found

In [ ]:
pref2pgn

In [ ]:
pref2pgn

In [ ]:
runpars

In [ ]:
pref2pgn

In [ ]:
#prefixes_types, pfstr_per_rnt ['normal']

In [ ]:
runstrings

In [ ]:
pref2pgn

In [ ]:
special_globinds

In [ ]:
len(runstrings)

In [ ]:
#special_globinds_mod = special_globinds
special_globinds_mod = [gi + n_testrunstr_to_include for gi in special_globinds]
#[runstrings[gi][120:-93] for gi in special_globinds_mod]
runstrings_sub = [runstrings[gi] for gi in special_globinds_mod]
display( runstrings_sub )
#special_globinds_mod
print( ','.join(map(str,special_globinds_mod) ) )

In [ ]:
len(runstrings)

In [ ]:
sis = []
for rsi,rs in enumerate(runstrings):
    #if rs.find('--mods LFP') >= 0:
    if rs.find('15') >= 0:
        sis += [rsi]
        print(rs)
print( ','.join(map(str,sis) ) )

In [ ]:
corresp

In [ ]:
runstrings_per_rnt['test' ]

In [ ]:
runstrings_per_rnt

In [ ]:
gv.data_dir

In [ ]:
pwd

In [ ]:
pref2pgn

In [ ]:
runstrings[0]

In [ ]:

#open()

In [ ]:
s